# Лабораторная работа №5
## Тема: Задача линейного программирования. Симплекс-метод.

In [1]:
from copy import copy
import numpy as np
import pandas as pd

In [2]:
from pprint import pprint
from IPython.display import display, HTML

#### Условие
$\begin{cases} 
    -2x_1 - x_2 + 5x_3 \rightarrow max \\
    -x_1 + x_2 - 2x_3 \ge 2 \\
    -4x_1 + 5x_2 - 4x_3 \ge 25 \\
    x_2 - x_3 \le 8 \\
    5x_1 - x_2 - x_3 \ge -1 \\
    3x_3 \ge 2 \\
    x_1, x_2, x_3 \ge 0
\end{cases}$

##### Привеведем к каноничному виду
$\begin{cases} 
    -2x_1 - x_2 + 5x_3 \rightarrow max \\
    -x_1 + x_2 - 2x_3 \ge 2 \\
    -4x_1 + 5x_2 - 4x_3 \ge 25 \\
    x_2 - x_3 \le 8 \\
    -5x_1 + x_2 + x_3 \le 1 \\
    3x_3 \ge 2 \\
    x_1, x_2, x_3 \ge 0
\end{cases}
\Rightarrow
\begin{cases} 
    -2x_1 - x_2 + 5x_3 - Mx_9 - Mx_{10} - Mx_{11} \rightarrow max \\
    -x_1 + x_2 - 2x_3 - x_4 + x_9 = 2 \\
    -4x_1 + 5x_2 - 4x_3 - x_5 + x_{10} = 25 \\
    x_2 - x_3 + x_6 = 8 \\
    -5x_1 + x_2 + x_3 + x_7 = 1 \\
    3x_3 - x_8 + x_{11} = 2 \\
    x_1, x_2, x_3 \ge 0
\end{cases} M\gg1
$

##### Выразим искусственные переменные $x_9, x_{10}, x_{11}$
$\begin{cases} 
    x_9 = 2 + x_1 - x_2 + 2x_3 + x_4 \\
    x_{10} = 25 + 4x_1 - 5x_2 + 4x_3 + x_5 \\
    x_{11} = 2 - 3x_3 + x_8
\end{cases}
$

##### Подставим выраженные искусственные переменные в целевую функцию
$
F(x) = -2x_1 - x_2 + 5x_3 - M(2 + x_1 - x_2 + 2x_3 + x_4) - M(25 + 4x_1 - 5x_2 + 4x_3 + x_5) - M(2 - 3x_3 + x_8) \\
F(x) = (-2 - 5M)x_1 + (-1 + 6M)x_2 + (5 - 3M)x_3 + (-M)x_4 + (-M)x_5 + (-M)x_6 + (-M)x_8 + (-29M)
$

In [3]:
def func(x):
    y = -2*x[0] - x[1] + 5*x[2]
    return y

In [4]:
M = 1e2

In [5]:
A0 = np.array([[-1, 1, -2],
               [-4, 5, -4],
               [ 0, 1, -1],
               [-5, 1,  1],
               [ 0, 0,  3]])
A1 = np.array([[-1,  0, 0, 0,  0],
               [ 0, -1, 0, 0,  0],
               [ 0,  0, 1, 0,  0],
               [ 0,  0, 0, 1,  0],
               [ 0,  0, 0, 0, -1]])
A2 = np.array([[1, 0, 0],
               [0, 1, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 1]])
B = np.array([[ 2],
              [25],
              [ 8],
              [ 1],
              [ 2]])
F = np.array([[-29*M, 2+5*M, 1-6*M, -5+3*M,  M,  M, 0, 0, M, 0, 0, 0]])

basis = np.array([8, 9, 5, 6, 10])  # Индексы x_k из базиса

In [6]:
A = A0
A = np.append(A, A1, axis=1)
A = np.append(A, A2, axis=1)

In [7]:
def check_index(f):
    return (f >= 0).all()

In [8]:
def find_new_basis(f):
    return np.argmin(f)

In [9]:
def find_leading_line(b, x_k):
    return np.argmin(b / np.where(x_k > 0, x_k, np.min(x_k[x_k>0])/100))
#     return np.argmin(b / np.maximum(x_k, 0))

In [10]:
def get_solving_element(matr, i, j):
    return matr[i][j]

In [11]:
def merge_table(a, b, f):
    table = np.append(b, a, axis=1)
    table = np.append(table, f, axis=0)
    return table

In [12]:
def run(A_start, B_start, F_start, Basis):
    a = np.copy(A_start)
    b = np.copy(B_start)
    f = np.copy(F_start)
    basis = np.copy(Basis)
    
    count = 0
    table_arr = []
    table_arr.append(pd.DataFrame(merge_table(a, b, f), 
                                 columns=['$B$'] + [f'$x_{{{i+1}}}$' for i in range(a.shape[1])],
                                 index=[f'$x_{{{i+1}}}$' for i in basis] + [f'$F(X_{{{count}}})$']))
    while True:
        if check_index(f.flatten()[1:]):
            break
        table = merge_table(a, b, f)
        j = find_new_basis(f.flatten()[1:])
        i = find_leading_line(b.flatten(), a[:, j])
        new_se = get_solving_element(a, i, j)
        
        res = np.copy(table)
        for n in range(res.shape[0]):
            for m in range(res.shape[1]):
                res[n][m] = table[n][m] - (table[i][m] * table[n][j+1]) / new_se
        for m in range(res.shape[1]):
            res[i][m] = table[i][m] / new_se
        
        basis[i] = j
        a = res[:-1, 1:]
        b = res[:-1, :1]
        f = res[-1:, :]
        count += 1
        table_arr.append(pd.DataFrame(res, 
                                     columns=['$B$'] + [f'$x_{{{i+1}}}$' for i in range(a.shape[1])],
                                     index=[f'$x_{{{i+1}}}$' for i in basis] + [f'$F(X_{{{count}}})$']))
        
    final = np.zeros(a.shape[1])
    for idx, val in enumerate(basis):
        final[val] = b[idx]
    
    solution = func(final)

    return solution, count, table_arr

In [13]:
answer, c, tables = run(A, B, F, basis)

In [14]:
print(f'Решение задачи: {answer}')

Решение задачи: -0.2857142857142847


In [15]:
print(f'Решение было получено за {c} итераций')

Решение было получено за 6 итераций


In [16]:
for t in tables:
    display(t)

,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{9}$,2.0,-1.0,1.0,-2.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
$x_{10}$,25.0,-4.0,5.0,-4.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0
$x_{6}$,8.0,0.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
$x_{7}$,1.0,-5.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
$x_{11}$,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0
$F(X_{0})$,-2900.0,502.0,-599.0,295.0,100.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{9}$,1.0,4.0,0.0,-3.0,-1.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0
$x_{10}$,20.0,21.0,0.0,-9.0,0.0,-1.0,0.0,-5.0,0.0,0.0,1.0,0.0
$x_{6}$,7.0,5.0,0.0,-2.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0
$x_{2}$,1.0,-5.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
$x_{11}$,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0
$F(X_{1})$,-2301.0,-2493.0,0.0,894.0,100.0,100.0,0.0,599.0,100.0,0.0,0.0,0.0


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{1}$,0.25,1.0,0.0,-0.75,-0.25,0.0,0.0,-0.25,0.0,0.25,0.0,0.0
$x_{10}$,14.75,0.0,0.0,6.75,5.25,-1.0,0.0,0.25,0.0,-5.25,1.0,0.0
$x_{6}$,5.75,0.0,0.0,1.75,1.25,0.0,1.0,0.25,0.0,-1.25,0.0,0.0
$x_{2}$,2.25,0.0,1.0,-2.75,-1.25,0.0,0.0,-0.25,0.0,1.25,0.0,0.0
$x_{11}$,2.00,0.0,0.0,3.00,0.00,0.0,0.0,0.00,-1.0,0.00,0.0,1.0
$F(X_{2})$,-1677.75,0.0,0.0,-975.75,-523.25,100.0,0.0,-24.25,100.0,623.25,0.0,0.0


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{1}$,0.750000,1.0,0.0,0.0,-0.25,0.0,0.0,-0.25,-0.250000,0.25,0.0,0.250000
$x_{10}$,10.250000,0.0,0.0,0.0,5.25,-1.0,0.0,0.25,2.250000,-5.25,1.0,-2.250000
$x_{6}$,4.583333,0.0,0.0,0.0,1.25,0.0,1.0,0.25,0.583333,-1.25,0.0,-0.583333
$x_{2}$,4.083333,0.0,1.0,0.0,-1.25,0.0,0.0,-0.25,-0.916667,1.25,0.0,0.916667
$x_{3}$,0.666667,0.0,0.0,1.0,0.00,0.0,0.0,0.00,-0.333333,0.00,0.0,0.333333
$F(X_{3})$,-1027.250000,0.0,0.0,0.0,-523.25,100.0,0.0,-24.25,-225.250000,623.25,0.0,325.250000


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{1}$,1.238095,1.0,0.0,0.0,0.0,-0.047619,0.0,-0.238095,-0.142857,0.0,0.047619,0.142857
$x_{4}$,1.952381,0.0,0.0,0.0,1.0,-0.190476,0.0,0.047619,0.428571,-1.0,0.190476,-0.428571
$x_{6}$,2.142857,0.0,0.0,0.0,0.0,0.238095,1.0,0.190476,0.047619,0.0,-0.238095,-0.047619
$x_{2}$,6.523810,0.0,1.0,0.0,0.0,-0.238095,0.0,-0.190476,-0.380952,0.0,0.238095,0.380952
$x_{3}$,0.666667,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,-0.333333,0.0,0.000000,0.333333
$F(X_{4})$,-5.666667,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,-1.000000,100.0,99.666667,101.000000


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{1}$,1.888889,1.0,0.0,0.0,0.333333,-0.111111,0.0,-0.222222,0.0,-0.333333,0.111111,0.0
$x_{8}$,4.555556,0.0,0.0,0.0,2.333333,-0.444444,0.0,0.111111,1.0,-2.333333,0.444444,-1.0
$x_{6}$,1.925926,0.0,0.0,0.0,-0.111111,0.259259,1.0,0.185185,0.0,0.111111,-0.259259,0.0
$x_{2}$,8.259259,0.0,1.0,0.0,0.888889,-0.407407,0.0,-0.148148,0.0,-0.888889,0.407407,0.0
$x_{3}$,2.185185,0.0,0.0,1.0,0.777778,-0.148148,0.0,0.037037,0.0,-0.777778,0.148148,0.0
$F(X_{5})$,-1.111111,0.0,0.0,0.0,2.333333,-0.111111,0.0,0.777778,0.0,97.666667,100.111111,100.0


,$B$,$x_{1}$,$x_{2}$,$x_{3}$,$x_{4}$,$x_{5}$,$x_{6}$,$x_{7}$,$x_{8}$,$x_{9}$,$x_{10}$,$x_{11}$
$x_{1}$,2.714286,1.0,0.0,0.0,0.285714,0.0,0.428571,-0.142857,0.0,-0.285714,0.0,0.0
$x_{8}$,7.857143,0.0,0.0,0.0,2.142857,0.0,1.714286,0.428571,1.0,-2.142857,0.0,-1.0
$x_{5}$,7.428571,0.0,0.0,0.0,-0.428571,1.0,3.857143,0.714286,0.0,0.428571,-1.0,0.0
$x_{2}$,11.285714,0.0,1.0,0.0,0.714286,0.0,1.571429,0.142857,0.0,-0.714286,0.0,0.0
$x_{3}$,3.285714,0.0,0.0,1.0,0.714286,0.0,0.571429,0.142857,0.0,-0.714286,0.0,0.0
$F(X_{6})$,-0.285714,0.0,0.0,0.0,2.285714,0.0,0.428571,0.857143,0.0,97.714286,100.0,100.0
